# COGS 108 - Data Checkpoint

# Names

- Jake Heinlein
- Nathan Tripp
- Naomi Chin
- Leo Friedman
- Dante Tanjuatco

<a id='research_question'></a>
# Research Question

*Is the combination of an mlb free agents age and batting performance, measured by batting average and on base plus slugging percentage, indicative of their yearly salary, and if so, can an algorithm predict a players future contract based on these factors?*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

The first data set looks at MLB players stats by year, which we are using to look at players age, batting average, and on base plus slugging percentage. The second dataset shows free agent contracts, which we will use to find players yearly salary.

We are looking at players who are included in both sets, comparing batting performance along with age to their yearly salary.

# Setup

In [42]:
import pandas as pd
pd.set_option('display.max_rows', None)
import os

In [50]:
def salary_to_int(str_in):
    str_in = str_in.replace('$','')
    str_in = str_in.replace(',','')
    output = int(str_in)
    return output

In [70]:
directory = 'data/contracts'
filenames = []
for filename in os.listdir(directory):
    filenames.append(str(directory) + "/" + filename)
contracts = pd.concat([pd.read_csv(f) for f in filenames])
contracts = contracts[["Player","Pos'n","Age","Term","AAV"]]
contracts = contracts[contracts["Pos'n"].str.contains("hp") == False]
contracts = contracts[contracts["AAV"].isna() == False]
contracts = contracts[contracts["AAV"] != "$1"]
contracts["AAV"] = contracts["AAV"].apply(salary_to_int)
contracts = contracts.reset_index(drop = True)
contracts = contracts.rename({"AAV":"Yearly Salary"}, axis=1)
display(contracts.head())

,Player,Pos'n,Age,Term,Yearly Salary
0,"Harper, Bryce",rf,26.0,2019-31,25384615
1,"Machado, Manny",ss,26.0,2019-28,30000000
2,"Pollock, A.J.",cf,31.0,2019-22,13750000
3,"McCutchen, Andrew",lf,32.0,2019-21,16666666
4,"Brantley, Michael",lf,32.0,2019-20,16000000


In [ ]:
directory = 'data/contracts'
filenames = []
for filename in os.listdir(directory):
    filenames.append(str(directory) + "/" + filename)
contracts = pd.concat([pd.read_csv(f) for f in filenames])

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION